# References and Resources 
* https://www.nytimes.com/2022/01/16/world/asia/tonga-tsunami-peru.html
* https://en.wikipedia.org/wiki/Tonga

**Eruption from satellite images**
![](https://cdn.cnn.com/cnnnext/dam/assets/220116193237-tonga-volcanic-eruption-explainer-super-tease.jpg)

**Sequence of Explosion**
![](https://i0.wp.com/media.techeblog.com/images/tonga-volcano-eruption-space.jpg?w=1200&ssl=1)

# Note:

    The layers on top of folium maps will dissapear in couple of days. Please re-run the notebook on your end to visualize the results.



# Imports

In [ ]:
import requests
import numpy as np 
import os 
import pickle
import pandas as pd
from collections import defaultdict

import ee 
import matplotlib.pyplot as plt 
import folium
import branca.colormap as cmp
from scipy import optimize
from folium.plugins import MarkerCluster,HeatMap,HeatMapWithTime
from IPython import display

# Authenticate and initialize EE

In [ ]:
ee.Authenticate()

In [ ]:
ee.Initialize()

# Helper functions

In [ ]:

# setup to add ee layer to folium

def add_ee_layer(self, ee_image_object, vis_params, name,opacity=0.75):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
      tiles=map_id_dict['tile_fetcher'].url_format,
      attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
      name=name,
      opacity=opacity,
      overlay=True,
      control=True
    ).add_to(self)

folium.Map.add_ee_layer = add_ee_layer


def show_layer_on_folium(
                        layer,
                        title,
                        viz_params,
                        location=None,
                        map_bounds=None,
                        colormap=None,
                        download=False,
                        opacity=0.90,
                        basemap= 'OpenStreetMap',
                        zoom_start=9,
                        max_zoom=None,
                        min_zoom=6):
    
    #initiate a map
    map_1=folium.Map(location,tiles=basemap,min_zoom=min_zoom,max_zoom=max_zoom,zoom_start=zoom_start)
    
    if map_bounds: 
        #fit map to our geometry
        folium.FitBounds(bounds=map_bounds).add_to(map_1)

    #add ee layer 
    map_1.add_ee_layer(ee_image_object=layer,
                       vis_params=viz_params,
                       name=title,
                      opacity=opacity)
    
    #add layer control
    map_1.add_child(folium.LayerControl())
    
    if download:
        map_1.save(f'{title}.html')
        return None
    else:
        return map_1


def get_coordinates(address,return_json=False,return_bb= False):
    '''Get lat-long for a given address
    SRC : https://stackoverflow.com/questions/25888396/how-to-get-latitude-longitude-with-python'''
    
    import requests
    from urllib import parse
    
    response = requests.get('https://nominatim.openstreetmap.org/search/' + parse.quote(address) + '?format=json')

    resp= response.json()
    
    #return complete response file 
    if return_json:
        return resp
    
    if return_bb:
        return [(float(resp[0]['boundingbox'][i]),float(resp[0]['boundingbox'][i+2])) for i in range(len(resp[0]['boundingbox'])//2)]
    
#   'returns LAT -  LONG'
    
    return [float(resp[0]['lat']),float(resp[0]['lon'])]



def get_best_image(image_collection,
                   year,
                   area_of_interest,
                   month=10):
    '''filter the best image( with the least cloud cover value) for the given year and given month'''

    start_date = ee.Date.fromYMD(**{
        'day':1 ,
        'month': month,
        'year' : year})
    end_date = ee.Date.fromYMD(**{
        'day':30 ,
        'month': month,
        'year' : year})
    
    
    if not area_of_interest:
        # sort image by asc order of cloud pixel values
        img = image_collection.filterDate(start_date,end_date).sort('CLOUD_COVER').first()
    else:
        img = image_collection.filterDate(start_date,end_date).sort('CLOUD_COVER').first().clip(area_of_interest)
        
    
    return img 

In [ ]:
#get location coordinates 
tonga_coord = get_coordinates('Tonga Volcano')

# international boundaries dataset
countries = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017')
#get geometry bounds for tonga
tonga_geom = countries.filter(ee.Filter.eq('country_na','Tonga'))

#boundaries for fitting map
tonga_bounds=[(i[1],i[0]) for i in tonga_geom.geometry().getInfo().get('coordinates')[0]]

area = ee.Number(tonga_geom.geometry().area()).divide(1e+6).round().getInfo()

print(f'Total administrative area of Tonga Islands is {area} Sqkm')

In [ ]:
#get a buffer of 1500 km around AOI 
buff= tonga_geom.geometry().buffer(1500000)

#location of volcano
vol = ee.geometry.Geometry.Point(tonga_coord).buffer(35000)

In [ ]:
import rasterio as rio  
from IPython.display import Image 



def scale(band):
    '''scale a image by max value'''
    return band/np.max(band)

img = rio.open('../input/tonga-image/859a7a8417c9b4d22e7c4d8a6c433d31-a17c86d9aa797e3c7693609b284f38ad getPixels.tif')


In [84]:
b=scale(img.read(1))
g=scale(img.read(2))
r=scale(img.read(3))

rgb_im = np.dstack((b,g,r))
#save image to display
plt.imsave('tonga_atoll.jpeg',rgb_im)

![](./tonga_atoll.jpeg)

In [87]:


m1= folium.Map(location=tonga_coord,zoom_start=9)

folium.CircleMarker(tonga_coord,radius=15,popup = '<b>Approx location of Tonga Volcano</b>').add_to(m1)

print('Approximate location to look into')

m1

Approximate location to look into


In [88]:
#dates to check for Images 
dates_bf = ('2022-01-01', '2022-01-14')
dates_af = ('2022-01-15', '2022-01-18')


# Visualizing Images 

# Checking the Sulphur Dioxide emissions around the area
    Volcanoes Emit a lot os SO2 into the atmosphere. We can expect the concentration of SO2 to go up drastically in and around the region.

In [89]:
#get the mean of so2 concentration layer

image1=ee.ImageCollection('COPERNICUS/S5P/NRTI/L3_SO2')\
  .select('SO2_column_number_density')\
  .filterDate(dates_bf[0],dates_bf[1]).mean().clip(buff)

image2=ee.ImageCollection('COPERNICUS/S5P/NRTI/L3_SO2')\
  .select('SO2_column_number_density')\
  .filterDate(dates_af[0],dates_af[1]).mean().clip(buff)


In [90]:

image_viz_params = {'min': - 0.0005, 'max': 0.005,
    'palette': ['black', 'blue', 'purple', 'cyan', 'green', 'yellow', 'red']
}
#initiate a dual map
map_1=folium.plugins.DualMap(layout='horizontal',location = tonga_coord,zoom_start=6)

#fit map to our geometry
# folium.FitBounds(bounds=tonga_bounds).add_to(map_1)

#add ee layer 
map_1.m1.add_ee_layer(ee_image_object=image1,
                     vis_params=image_viz_params,
                     name='Mean SO2 Concentration',
                     opacity=0.75)


#add ee layer 
map_1.m2.add_ee_layer(ee_image_object=image2,
                     vis_params=image_viz_params,
                     name='Mean SO2 Concentration',
                     opacity=0.75)

folium.Marker(tonga_coord,popup = '<b>Approx location of Tonga</b>').add_to(map_1.m1)
folium.Marker(tonga_coord,popup = '<b>Approx location of Tonga</b>').add_to(map_1.m2)



cbar = cmp.StepColormap(
 image_viz_params['palette'],
 vmin=image_viz_params['min'], vmax=image_viz_params['max'],
 caption='SO2 Concentration'
)

#add colorbar
cbar.add_to(map_1.m1)


#add layer control
map_1.add_child(folium.LayerControl())


print('Sulphur dioxide layer (left-before,right-after)')

display.display_html(map_1)

Sulphur dioxide layer (left-before,right-after)


Make this Notebook Trusted to load map: File -> Trust Notebook <iframe src="about:blank" style="position:absolute;width:100%;height:100%;left:0;top:0;border:none !important;" data-html=%3C%21DOCTYPE%20html%3E%0A%3Chead%3E%20%20%20%20%0A%20%20%20%20%3Cmeta%20http-equiv%3D%22content-type%22%20content%3D%22text/html%3B%20charset%3DUTF-8%22%20/%3E%0A%20%20%20%20%0A%20%20%20%20%20%20%20%20%3Cscript%3E%0A%20%20%20%20%20%20%20%20%20%20%20%20L_NO_TOUCH%20%3D%20false%3B%0A%20%20%20%20%20%20%20%20%20%20%20%20L_DISABLE_3D%20%3D%20false%3B%0A%20%20%20%20%20%20%20%20%3C/script%3E%0A%20%20%20%20%0A%20%20%20%20%3Cstyle%3Ehtml%2C%20body%20%7Bwidth%3A%20100%25%3Bheight%3A%20100%25%3Bmargin%3A%200%3Bpadding%3A%200%3B%7D%3C/style%3E%0A%20%20%20%20%3Cstyle%3E%23map%20%7Bposition%3Aabsolute%3Btop%3A0%3Bbottom%3A0%3Bright%3A0%3Bleft%3A0%3B%7D%3C/style%3E%0A%20%20%20%20%3Cscript%20src%3D%22https%3A//cdn.jsdelivr.net/npm/leaflet%401.6.0/dist/leaflet.js%22%3E%3C/script%3E%0A%20%20%20%20%3Cscript%20src%3D%22https%3A//code.jquery.com/jquery-1.12.4.min.js%22%3E%3C/script%3E%0A%20%20%20%20%3Cscript%20src%3D%22https%3A//maxcdn.bootstrapcdn.com/bootstrap/3.2.0/js/bootstrap.min.js%22%3E%3C/script%3E%0A%20%20%20%20%3Cscript%20src%3D%22https%3A//cdnjs.cloudflare.com/ajax/libs/Leaflet.awesome-markers/2.0.2/leaflet.awesome-markers.js%22%3E%3C/script%3E%0A%20%20%20%20%3Clink%20rel%3D%22stylesheet%22%20href%3D%22https%3A//cdn.jsdelivr.net/npm/leaflet%401.6.0/dist/leaflet.css%22/%3E%0A%20%20%20%20%3Clink%20rel%3D%22stylesheet%22%20href%3D%22https%3A//maxcdn.bootstrapcdn.com/bootstrap/3.2.0/css/bootstrap.min.css%22/%3E%0A%20%20%20%20%3Clink%20rel%3D%22stylesheet%22%20href%3D%22https%3A//maxcdn.bootstrapcdn.com/bootstrap/3.2.0/css/bootstrap-theme.min.css%22/%3E%0A%20%20%20%20%3Clink%20rel%3D%22stylesheet%22%20href%3D%22https%3A//maxcdn.bootstrapcdn.com/font-awesome/4.6.3/css/font-awesome.min.css%22/%3E%0A%20%20%20%20%3Clink%20rel%3D%22stylesheet%22%20href%3D%22https%3A//cdnjs.cloudflare.com/ajax/libs/Leaflet.awesome-markers/2.0.2/leaflet.awesome-markers.css%22/%3E%0A%20%20%20%20%3Clink%20rel%3D%22stylesheet%22%20href%3D%22https%3A//cdn.jsdelivr.net/gh/python-visualization/folium/folium/templates/leaflet.awesome.rotate.min.css%22/%3E%0A%20%20%20%20%0A%20%20%20%20%20%20%20%20%20%20%20%20%3Cmeta%20name%3D%22viewport%22%20content%3D%22width%3Ddevice-width%2C%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20initial-scale%3D1.0%2C%20maximum-scale%3D1.0%2C%20user-scalable%3Dno%22%20/%3E%0A%20%20%20%20%20%20%20%20%20%20%20%20%3Cstyle%3E%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%23map_5e808b7457134951944738a951bffc7a%20%7B%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20position%3A%20absolute%3B%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20width%3A%2050.0%25%3B%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20height%3A%20100.0%25%3B%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20left%3A%200.0%25%3B%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20top%3A%200.0%25%3B%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%7D%0A%20%20%20%20%20%20%20%20%20%20%20%20%3C/style%3E%0A%20%20%20%20%20%20%20%20%0A%20%20%20%20%3Cscript%20src%3D%22https%3A//cdnjs.cloudflare.com/ajax/libs/d3/3.5.5/d3.min.js%22%3E%3C/script%3E%0A%20%20%20%20%0A%20%20%20%20%20%20%20%20%20%20%20%20%3Cmeta%20name%3D%22viewport%22%20content%3D%22width%3Ddevice-width%2C%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20initial-scale%3D1.0%2C%20maximum-scale%3D1.0%2C%20user-scalable%3Dno%22%20/%3E%0A%20%20%20%20%20%20%20%20%20%20%20%20%3Cstyle%3E%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%23map_47bbb03fb3b240cebd25cebbe5ac4d01%20%7B%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20position%3A%20absolute%3B%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20width%3A%2050.0%25%3B%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20height%3A%20100.0%25%3B%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%

# Checking Dust and Aerosol Emissions


**Sentinel 5p: Aerosol Index**

    Absorbing aerosols, such as smoke from biomass burning, desert dust, volcanic ash, and anthropogenically produced soot, absorb radiation and have a warming effect on the climate. Scattering aerosols, like sulfate particles and clouds, scatter solar light and usually have a cooling effect on the climate. Aerosols also act as condensation nuclei in the process of cloud formation, potentially altering the optical properties of these clouds.
    
    
    The AAI is traditionally defined as the positive values of the reflectance residue between an absorbing-aerosol-loaded atmosphere and a clear atmosphere. Negative values are associated with an atmosphere that contains more scattering particles than a clear atmosphere. SRC: https://amt.copernicus.org/articles/13/6407/2020/



In [91]:
#get the mean of aerosol index layer

im1=ee.ImageCollection("COPERNICUS/S5P/NRTI/L3_AER_AI").\
                            select('absorbing_aerosol_index').\
                            filterDate('2022-01-10', '2022-01-14').mean().clip(buff)

im2=ee.ImageCollection("COPERNICUS/S5P/NRTI/L3_AER_AI").\
                            select('absorbing_aerosol_index').\
                            filterDate('2022-01-15', '2022-01-18').mean().clip(buff)


In [92]:

image_viz_params = {'min': -1, 'max': 1,
    'palette': ['black', 'blue', 'purple', 'cyan', 'green', 'yellow', 'red']
}
#initiate a dual map
map_1=folium.plugins.DualMap(layout='horizontal',location = tonga_coord,zoom_start=6)

#fit map to our geometry
# folium.FitBounds(bounds=tonga_bounds).add_to(map_1)

#add ee layer 
map_1.m1.add_ee_layer(ee_image_object=im1,
                     vis_params=image_viz_params,
                     name='Mean SO2 Concentration',
                     opacity=0.75)


#add ee layer 
map_1.m2.add_ee_layer(ee_image_object=im2,
                     vis_params=image_viz_params,
                     name='Mean SO2 Concentration',
                     opacity=0.75)

folium.Marker(tonga_coord,popup = '<b>Approx location of Volcano</b>').add_to(map_1.m1)
folium.Marker(tonga_coord,popup = '<b>Approx location of Volcano</b>').add_to(map_1.m2)



cbar = cmp.StepColormap(
 image_viz_params['palette'],
 vmin=image_viz_params['min'], vmax=image_viz_params['max'],
 caption='Aerosol Index '
)

#add colorbar
cbar.add_to(map_1.m1)


#add layer control
map_1.add_child(folium.LayerControl())

print('Mean Aerosol Index layer (left-before,right-after)')

display.display_html(map_1)

Mean Aerosol Index layer (left-before,right-after)


Make this Notebook Trusted to load map: File -> Trust Notebook <iframe src="about:blank" style="position:absolute;width:100%;height:100%;left:0;top:0;border:none !important;" data-html=%3C%21DOCTYPE%20html%3E%0A%3Chead%3E%20%20%20%20%0A%20%20%20%20%3Cmeta%20http-equiv%3D%22content-type%22%20content%3D%22text/html%3B%20charset%3DUTF-8%22%20/%3E%0A%20%20%20%20%0A%20%20%20%20%20%20%20%20%3Cscript%3E%0A%20%20%20%20%20%20%20%20%20%20%20%20L_NO_TOUCH%20%3D%20false%3B%0A%20%20%20%20%20%20%20%20%20%20%20%20L_DISABLE_3D%20%3D%20false%3B%0A%20%20%20%20%20%20%20%20%3C/script%3E%0A%20%20%20%20%0A%20%20%20%20%3Cstyle%3Ehtml%2C%20body%20%7Bwidth%3A%20100%25%3Bheight%3A%20100%25%3Bmargin%3A%200%3Bpadding%3A%200%3B%7D%3C/style%3E%0A%20%20%20%20%3Cstyle%3E%23map%20%7Bposition%3Aabsolute%3Btop%3A0%3Bbottom%3A0%3Bright%3A0%3Bleft%3A0%3B%7D%3C/style%3E%0A%20%20%20%20%3Cscript%20src%3D%22https%3A//cdn.jsdelivr.net/npm/leaflet%401.6.0/dist/leaflet.js%22%3E%3C/script%3E%0A%20%20%20%20%3Cscript%20src%3D%22https%3A//code.jquery.com/jquery-1.12.4.min.js%22%3E%3C/script%3E%0A%20%20%20%20%3Cscript%20src%3D%22https%3A//maxcdn.bootstrapcdn.com/bootstrap/3.2.0/js/bootstrap.min.js%22%3E%3C/script%3E%0A%20%20%20%20%3Cscript%20src%3D%22https%3A//cdnjs.cloudflare.com/ajax/libs/Leaflet.awesome-markers/2.0.2/leaflet.awesome-markers.js%22%3E%3C/script%3E%0A%20%20%20%20%3Clink%20rel%3D%22stylesheet%22%20href%3D%22https%3A//cdn.jsdelivr.net/npm/leaflet%401.6.0/dist/leaflet.css%22/%3E%0A%20%20%20%20%3Clink%20rel%3D%22stylesheet%22%20href%3D%22https%3A//maxcdn.bootstrapcdn.com/bootstrap/3.2.0/css/bootstrap.min.css%22/%3E%0A%20%20%20%20%3Clink%20rel%3D%22stylesheet%22%20href%3D%22https%3A//maxcdn.bootstrapcdn.com/bootstrap/3.2.0/css/bootstrap-theme.min.css%22/%3E%0A%20%20%20%20%3Clink%20rel%3D%22stylesheet%22%20href%3D%22https%3A//maxcdn.bootstrapcdn.com/font-awesome/4.6.3/css/font-awesome.min.css%22/%3E%0A%20%20%20%20%3Clink%20rel%3D%22stylesheet%22%20href%3D%22https%3A//cdnjs.cloudflare.com/ajax/libs/Leaflet.awesome-markers/2.0.2/leaflet.awesome-markers.css%22/%3E%0A%20%20%20%20%3Clink%20rel%3D%22stylesheet%22%20href%3D%22https%3A//cdn.jsdelivr.net/gh/python-visualization/folium/folium/templates/leaflet.awesome.rotate.min.css%22/%3E%0A%20%20%20%20%0A%20%20%20%20%20%20%20%20%20%20%20%20%3Cmeta%20name%3D%22viewport%22%20content%3D%22width%3Ddevice-width%2C%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20initial-scale%3D1.0%2C%20maximum-scale%3D1.0%2C%20user-scalable%3Dno%22%20/%3E%0A%20%20%20%20%20%20%20%20%20%20%20%20%3Cstyle%3E%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%23map_d46cbeb7a55e403c8c0f1e2544d78754%20%7B%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20position%3A%20absolute%3B%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20width%3A%2050.0%25%3B%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20height%3A%20100.0%25%3B%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20left%3A%200.0%25%3B%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20top%3A%200.0%25%3B%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%7D%0A%20%20%20%20%20%20%20%20%20%20%20%20%3C/style%3E%0A%20%20%20%20%20%20%20%20%0A%20%20%20%20%3Cscript%20src%3D%22https%3A//cdnjs.cloudflare.com/ajax/libs/d3/3.5.5/d3.min.js%22%3E%3C/script%3E%0A%20%20%20%20%0A%20%20%20%20%20%20%20%20%20%20%20%20%3Cmeta%20name%3D%22viewport%22%20content%3D%22width%3Ddevice-width%2C%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20initial-scale%3D1.0%2C%20maximum-scale%3D1.0%2C%20user-scalable%3Dno%22%20/%3E%0A%20%20%20%20%20%20%20%20%20%20%20%20%3Cstyle%3E%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%23map_37a98d116eae45b8bb6f8651e22bc18b%20%7B%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20position%3A%20absolute%3B%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20width%3A%2050.0%25%3B%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20height%3A%20100.0%25%3B%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%